# Loading the data

In [ ]:
!pip install espn_api

In [ ]:
import pandas as pd
import numpy as np
from espn_api.football import League

Good video on how to find league_id, espn_s2, and swid [here](https://www.youtube.com/watch?v=tNcND9lVycA&t=414s)

In [ ]:
league = League(league_id = 1233456789, # replace
                            year = 2025,
                            espn_s2 = "xyz", # replace
                            swid = "{xyz") # replace

# Creating df

In [ ]:
def pick_num(round, pick):
  """
  Function that takes draft round number and pick number (of that round) as inputs and returns overall draft pick number
  """
  round = round - 1

  if round == 0:
    pick_num = pick
  elif pick == 10:
    pick_num = int(str(round+1) + str(0))
  else:
    pick_num = int(str(round) + str(pick))

  return pick_num

In [ ]:
players = []
draft_picks = []
team_name = []

for pick in league.draft:
  players.append(pick.playerName)
  draft_picks.append(pick_num(pick.round_num, pick.round_pick))
  team_name.append(pick.team.team_name)

full_draft_df = pd.DataFrame({
    'Player': players,
    'Pick': draft_picks,
    'Team': team_name
    })


all_players = []
positions = []

for team in league.teams:
  for player in team.roster:
    all_players.append(player.name)
    positions.append(player.position)

full_pos_df = pd.DataFrame({
    'Player': all_players,
    'Position': positions
    })


df = pd.merge(full_draft_df, full_pos_df, on='Player', how='inner')

# Evaluating draft value across teams

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Superflex

In [ ]:
superflex_rankings = pd.read_csv("/content/drive/MyDrive/Fantasy Football/Draft value project/FP_2025_Superflex_Rankings.csv")

# remove unncessary columns
superflex_rankings = superflex_rankings.drop(columns=["TIERS", "TEAM", "POS", "BYE WEEK", "SOS SEASON", "ECR VS. ADP"])

# rename columns
superflex_rankings.columns.values[0] = 'Rank'
superflex_rankings.columns.values[1] = 'Player'

# rename Patrick Mahomes
superflex_rankings['Player'] = superflex_rankings['Player'].apply(
    lambda x: "Patrick Mahomes" if "patrick mahomes" in str(x).lower() else x
)

# make sure 'Player' column perfectly aligns with other df
superflex_rankings = superflex_rankings.rename(columns={
    superflex_rankings.columns[superflex_rankings.columns.str.contains("Player")][0]: "Player"
})

# # Exact match check
# print("In df:", "Player" in df.columns)
# print("In superflex_rankings:", "Player" in superflex_rankings.columns)

In [ ]:
superflex_value = pd.merge(df, superflex_rankings, on='Player', how='inner')

In [ ]:
superflex_value['Superflex Value'] = superflex_value['Pick'] - superflex_value['Rank']

In [ ]:
decay_rate = 0.02
superflex_value["Superflex Adjusted Value"] = round((superflex_value['Pick'] - superflex_value['Rank']) * (np.exp(-decay_rate * superflex_value['Pick'])), 1)

In [ ]:
superflex_example = superflex_value.groupby('Team')[['Superflex Value', 'Superflex Adjusted Value']].sum().reset_index()

## Standard

In [ ]:
standard_rankings = pd.read_csv("/content/drive/MyDrive/Fantasy Football/Draft value project/FP_2025_Standard_Rankings.csv")

# remove unncessary columns
standard_rankings = standard_rankings.drop(columns=["TIERS", "TEAM", "POS", "BYE WEEK", "SOS SEASON", "ECR VS. ADP"])

# rename columns
standard_rankings.columns.values[0] = 'Rank'
standard_rankings.columns.values[1] = 'Player'

# rename Patrick Mahomes
standard_rankings['Player'] = standard_rankings['Player'].apply(
    lambda x: "Patrick Mahomes" if "patrick mahomes" in str(x).lower() else x
)

# make sure 'Player' column perfectly aligns with other df
standard_rankings = standard_rankings.rename(columns={
    standard_rankings.columns[standard_rankings.columns.str.contains("Player")][0]: "Player"
})

In [ ]:
standard_value = pd.merge(df, standard_rankings, on='Player', how='inner')

In [ ]:
standard_value['Standard Value'] = standard_value['Pick'] - standard_value['Rank']

In [ ]:
standard_no_kicker = standard_value[standard_value["Position"] != "K"]

In [ ]:
decay = 0.02
standard_no_kicker["Standard Adjusted Value"] = round((standard_no_kicker['Pick'] - standard_no_kicker['Rank']) * (np.exp(-decay * standard_no_kicker['Pick'])), 1)

/tmp/ipython-input-2468428074.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  standard_no_kicker["Standard Adjusted Value"] = round((standard_no_kicker['Pick'] - standard_no_kicker['Rank']) * (np.exp(-decay * standard_no_kicker['Pick'])), 1)


In [ ]:
standard_example = standard_no_kicker.groupby('Team')[['Standard Value', 'Standard Adjusted Value']].sum().reset_index()

# Project

In [ ]:
!pip install great_tables

In [ ]:
from great_tables import GT, style, loc
import polars
import polars.selectors as cs

In [ ]:
my_team_superflex = superflex_value[superflex_value["Team"] == "Team Kyle"].drop(columns=["Team", "Superflex Adjusted Value"])

cols_1 = list(my_team_superflex.columns)
cols_1[1], cols_1[2] = cols_1[2], cols_1[1]  # swap first two
my_team_superflex = my_team_superflex[cols_1]

my_team_superflex

,Player,Position,Pick,Rank,Superflex Value
3,Bijan Robinson,RB,4,8,-4
16,Brian Thomas Jr.,WR,17,26,-9
23,Kyren Williams,RB,24,42,-18
36,Patrick Mahomes,QB,37,7,30
43,TreVeyon Henderson,RB,44,73,-29
56,Davante Adams,WR,57,50,7
63,Brock Purdy,QB,64,25,39
76,DJ Moore,WR,77,60,17
82,Rashee Rice,WR,84,76,8
93,Jameson Williams,WR,97,70,27


In [ ]:
(
  GT(my_team_superflex)
  .tab_header(
      title="Evaluating my draft picks using superflex value",
      subtitle="Good value picks colored green, reaches colored red"
  )
  .tab_stub(rowname_col="Player")
  .tab_stubhead(label="Player")
  .fmt_number(columns="Superflex Value", decimals=0)
  .tab_source_note(source_note="Draft Data: ESPN Fantasy Football")
  .tab_source_note(source_note="Value Data: FantasyPros")
  .tab_style(style=style.fill(color="#f37b80"), locations=loc.body(rows=["Kyren Williams", "TreVeyon Henderson"]))
  .tab_style(style=style.fill(color="#65ca8d"), locations=loc.body(rows=["Patrick Mahomes", "Brock Purdy", "Caleb Williams", "Jaylen Warren"]))
  .tab_style(style=style.fill(color="#f37b80"),locations=loc.body(columns="Player", rows=[2,4]))
  .tab_style(style=style.fill(color="#65ca8d"),locations=loc.body(columns="Player", rows=[3,6,10,13]))
  .tab_style(style=style.text(weight="bold"),locations=loc.body(columns="Superflex Value", rows=[2,3,4,6,10,13]))
)

GT(_tbl_data=                 Player Position  Pick  Rank  Superflex Value
3        Bijan Robinson       RB     4     8               -4
16     Brian Thomas Jr.       WR    17    26               -9
23       Kyren Williams       RB    24    42              -18
36      Patrick Mahomes       QB    37     7               30
43   TreVeyon Henderson       RB    44    73              -29
56        Davante Adams       WR    57    50                7
63          Brock Purdy       QB    64    25               39
76             DJ Moore       WR    77    60               17
82          Rashee Rice       WR    84    76                8
93     Jameson Williams       WR    97    70               27
99       Caleb Williams       QB   104    29               75
106        Tucker Kraft       TE   117   113                4
112        Emeka Egbuka       WR   124   122                2
124       Jaylen Warren       RB   137   102               35
127        Tyler Warren       TE   144   126               18, _body=<great_tables._gt_data.Body object at 0x7ef8c5c49a30>, _boxhead=Boxhead([ColInfo(var='Player', type=<ColInfoTypeEnum.stub: 2>, column_label='Player', column_align='left', column_width=None), ColInfo(var='Position', type=<ColInfoTypeEnum.default: 1>, column_label='Position', column_align='left', column_width=None), ColInfo(var='Pick', type=<ColInfoTypeEnum.default: 1>, column_label='Pick', column_align='right', column_width=None), ColInfo(var='Rank', type=<ColInfoTypeEnum.default: 1>, column_label='Rank', column_align='right', column_width=None), ColInfo(var='Superflex Value', type=<ColInfoTypeEnum.default: 1>, column_label='Superflex Value', column_align='right', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x7ef8c5c49910>, _spanners=Spanners([]), _heading=Heading(title='Evaluating my draft picks using superflex value', subtitle='Good value picks colored green, reaches colored red', preheader=None), _stubhead='Player', _source_notes=['Draft Data: ESPN Fantasy Football', 'Value Data: FantasyPros'], _footnotes=[], _styles=[StyleInfo(locname=LocBody(columns=None, rows=['Kyren Williams', 'TreVeyon Henderson'], mask=None), grpname=None, colname='Position', rownum=2, colnum=None, styles=[CellStyleFill(color='#f37b80')]), StyleInfo(locname=LocBody(columns=None, rows=['Kyren Williams', 'TreVeyon Henderson'], mask=None), grpname=None, colname='Position', rownum=4, colnum=None, styles=[CellStyleFill(color='#f37b80')]), StyleInfo(locname=LocBody(columns=None, rows=['Kyren Williams', 'TreVeyon Henderson'], mask=None), grpname=None, colname='Pick', rownum=2, colnum=None, styles=[CellStyleFill(color='#f37b80')]), StyleInfo(locname=LocBody(columns=None, rows=['Kyren Williams', 'TreVeyon Henderson'], mask=None), grpname=None, colname='Pick', rownum=4, colnum=None, styles=[CellStyleFill(color='#f37b80')]), StyleInfo(locname=LocBody(columns=None, rows=['Kyren Williams', 'TreVeyon Henderson'], mask=None), grpname=None, colname='Rank', rownum=2, colnum=None, styles=[CellStyleFill(color='#f37b80')]), StyleInfo(locname=LocBody(columns=None, rows=['Kyren Williams', 'TreVeyon Henderson'], mask=None), grpname=None, colname='Rank', rownum=4, colnum=None, styles=[CellStyleFill(color='#f37b80')]), StyleInfo(locname=LocBody(columns=None, rows=['Kyren Williams', 'TreVeyon Henderson'], mask=None), grpname=None, colname='Superflex Value', rownum=2, colnum=None, styles=[CellStyleFill(color='#f37b80')]), StyleInfo(locname=LocBody(columns=None, rows=['Kyren Williams', 'TreVeyon Henderson'], mask=None), grpname=None, colname='Superflex Value', rownum=4, colnum=None, styles=[CellStyleFill(color='#f37b80')]), StyleInfo(locname=LocBody(columns=None, rows=['Patrick Mahomes', 'Brock Purdy', 'Caleb Williams', 'Jaylen Warren'], mask=None), grpname=None, colname='Position', rownum=3, colnum=None, styles=[CellStyleFill(color='#65ca8d')]), StyleInfo(locname=LocBody(columns=None, rows=['Patrick Mahomes', 'Brock Purdy', 'Caleb Williams', 'Jaylen Warren'], mask=

In [ ]:
my_team_standard = standard_no_kicker[standard_no_kicker["Team"] == "Team Kyle"].drop(columns=["Team", "Standard Adjusted Value"])

cols_2 = list(my_team_standard.columns)
cols_2[1], cols_2[2] = cols_2[2], cols_2[1]  # swap first two
my_team_standard = my_team_standard[cols_2]

my_team_standard

,Player,Position,Pick,Rank,Standard Value
3,Bijan Robinson,RB,4,2,2
16,Brian Thomas Jr.,WR,17,13,4
23,Kyren Williams,RB,24,25,-1
36,Patrick Mahomes,QB,37,65,-28
43,TreVeyon Henderson,RB,44,53,-9
56,Davante Adams,WR,57,34,23
63,Brock Purdy,QB,64,87,-23
76,DJ Moore,WR,77,44,33
83,Rashee Rice,WR,84,57,27
94,Jameson Williams,WR,97,51,46


In [ ]:
(
  GT(my_team_standard)
  .tab_header(
      title="Evaluating my draft picks using standard value",
      subtitle="Good value picks colored green, reaches colored red"
  )
  .tab_stub(rowname_col="Player")
  .tab_stubhead(label="Player")
  .fmt_number(columns="Standard Value", decimals=0)
  .tab_source_note(source_note="Draft Data: ESPN Fantasy Football")
  .tab_source_note(source_note="Value Data: FantasyPros")
  .tab_style(style=style.fill(color="#f37b80"), locations=loc.body(rows=["Patrick Mahomes", "Brock Purdy"]))
  .tab_style(style=style.fill(color="#65ca8d"), locations=loc.body(rows=["DJ Moore", "Rashee Rice", "Jameson Williams", "Jaylen Warren"]))
  .tab_style(style=style.fill(color="#f37b80"),locations=loc.body(columns="Player", rows=[3,6]))
  .tab_style(style=style.fill(color="#65ca8d"),locations=loc.body(columns="Player", rows=[7,8,9,13]))
  .tab_style(style=style.text(weight="bold"),locations=loc.body(columns="Standard Value", rows=[3,6,7,8,9,13]))
)

GT(_tbl_data=                 Player Position  Pick  Rank  Standard Value
3        Bijan Robinson       RB     4     2               2
16     Brian Thomas Jr.       WR    17    13               4
23       Kyren Williams       RB    24    25              -1
36      Patrick Mahomes       QB    37    65             -28
43   TreVeyon Henderson       RB    44    53              -9
56        Davante Adams       WR    57    34              23
63          Brock Purdy       QB    64    87             -23
76             DJ Moore       WR    77    44              33
83          Rashee Rice       WR    84    57              27
94     Jameson Williams       WR    97    51              46
100      Caleb Williams       QB   104   101               3
109        Tucker Kraft       TE   117    97              20
115        Emeka Egbuka       WR   124   108              16
128       Jaylen Warren       RB   137    84              53
134        Tyler Warren       TE   144   114              30, _body=<great_tables._gt_data.Body object at 0x7ef8c5281910>, _boxhead=Boxhead([ColInfo(var='Player', type=<ColInfoTypeEnum.stub: 2>, column_label='Player', column_align='left', column_width=None), ColInfo(var='Position', type=<ColInfoTypeEnum.default: 1>, column_label='Position', column_align='left', column_width=None), ColInfo(var='Pick', type=<ColInfoTypeEnum.default: 1>, column_label='Pick', column_align='right', column_width=None), ColInfo(var='Rank', type=<ColInfoTypeEnum.default: 1>, column_label='Rank', column_align='right', column_width=None), ColInfo(var='Standard Value', type=<ColInfoTypeEnum.default: 1>, column_label='Standard Value', column_align='right', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x7ef8c5282d80>, _spanners=Spanners([]), _heading=Heading(title='Evaluating my draft picks using standard value', subtitle='Good value picks colored green, reaches colored red', preheader=None), _stubhead='Player', _source_notes=['Draft Data: ESPN Fantasy Football', 'Value Data: FantasyPros'], _footnotes=[], _styles=[StyleInfo(locname=LocBody(columns=None, rows=['Patrick Mahomes', 'Brock Purdy'], mask=None), grpname=None, colname='Position', rownum=3, colnum=None, styles=[CellStyleFill(color='#f37b80')]), StyleInfo(locname=LocBody(columns=None, rows=['Patrick Mahomes', 'Brock Purdy'], mask=None), grpname=None, colname='Position', rownum=6, colnum=None, styles=[CellStyleFill(color='#f37b80')]), StyleInfo(locname=LocBody(columns=None, rows=['Patrick Mahomes', 'Brock Purdy'], mask=None), grpname=None, colname='Pick', rownum=3, colnum=None, styles=[CellStyleFill(color='#f37b80')]), StyleInfo(locname=LocBody(columns=None, rows=['Patrick Mahomes', 'Brock Purdy'], mask=None), grpname=None, colname='Pick', rownum=6, colnum=None, styles=[CellStyleFill(color='#f37b80')]), StyleInfo(locname=LocBody(columns=None, rows=['Patrick Mahomes', 'Brock Purdy'], mask=None), grpname=None, colname='Rank', rownum=3, colnum=None, styles=[CellStyleFill(color='#f37b80')]), StyleInfo(locname=LocBody(columns=None, rows=['Patrick Mahomes', 'Brock Purdy'], mask=None), grpname=None, colname='Rank', rownum=6, colnum=None, styles=[CellStyleFill(color='#f37b80')]), StyleInfo(locname=LocBody(columns=None, rows=['Patrick Mahomes', 'Brock Purdy'], mask=None), grpname=None, colname='Standard Value', rownum=3, colnum=None, styles=[CellStyleFill(color='#f37b80')]), StyleInfo(locname=LocBody(columns=None, rows=['Patrick Mahomes', 'Brock Purdy'], mask=None), grpname=None, colname='Standard Value', rownum=6, colnum=None, styles=[CellStyleFill(color='#f37b80')]), StyleInfo(locname=LocBody(columns=None, rows=['DJ Moore', 'Rashee Rice', 'Jameson Williams', 'Jaylen Warren'], mask=None), grpname=None, colname='Position', rownum=7, colnum=None, styles=[CellStyleFill(color='#65ca8d')]), StyleInfo(locname=LocBody(columns=None, rows=['DJ Moore', 'Rashee Rice', 'Jameson Williams', 'Jaylen Warren'], mask=None), grpname=None, colname='Position', rownum=8, colnum=None, styles=[CellSty

In [ ]:
my_team_superflex_adj_value = superflex_value[superflex_value["Team"] == "Team Kyle"].drop(columns=["Team", "Superflex Value", "Rank"])
my_team_standard_adj_value = standard_no_kicker[standard_no_kicker["Team"] == "Team Kyle"].drop(columns=["Team", "Standard Value", "Rank"])

In [ ]:
my_team_adj_value = pd.merge(my_team_standard_adj_value, my_team_superflex_adj_value, on=['Player', 'Pick', 'Position'], how='inner')

cols = list(my_team_adj_value.columns)
cols[1], cols[2] = cols[2], cols[1]  # swap first two
my_team_adj_value = my_team_adj_value[cols]

my_team_adj_value

,Player,Position,Pick,Standard Adjusted Value,Superflex Adjusted Value
0,Bijan Robinson,RB,4,1.8,-3.7
1,Brian Thomas Jr.,WR,17,2.8,-6.4
2,Kyren Williams,RB,24,-0.6,-11.1
3,Patrick Mahomes,QB,37,-13.4,14.3
4,TreVeyon Henderson,RB,44,-3.7,-12.0
5,Davante Adams,WR,57,7.4,2.2
6,Brock Purdy,QB,64,-6.4,10.8
7,DJ Moore,WR,77,7.1,3.6
8,Rashee Rice,WR,84,5.0,1.5
9,Jameson Williams,WR,97,6.6,3.9


In [ ]:
(
  GT(my_team_adj_value)
  .tab_header(
      title="Evaluating my draft picks using adjusted value",
      subtitle="Good value picks colored green, reaches colored red"
  )
  .tab_stub(rowname_col="Player")
  .tab_stubhead(label="Player")
  .fmt_number(columns="Superflex Value", decimals=0)
  .tab_source_note(source_note="Draft Data: ESPN Fantasy Football")
  .tab_source_note(source_note="Value Data: FantasyPros")
  .tab_style(style=style.fill(color="#f37b80"), locations=loc.body(rows=["TreVeyon Henderson"]))
  .tab_style(style=style.fill(color="#65ca8d"), locations=loc.body(rows=["DJ Moore", "Jameson Williams", "Jaylen Warren"]))
  .tab_style(style=style.fill(color="#f37b80"),locations=loc.body(columns="Player", rows=[4]))
  .tab_style(style=style.fill(color="#65ca8d"),locations=loc.body(columns="Player", rows=[7,9,13]))
  .tab_style(style=style.text(weight="bold"),locations=loc.body(columns=["Standard Adjusted Value", "Superflex Adjusted Value"], rows=[4,7,9,13]))
)

GT(_tbl_data=                Player Position  Pick  Standard Adjusted Value  \
0       Bijan Robinson       RB     4                      1.8   
1     Brian Thomas Jr.       WR    17                      2.8   
2       Kyren Williams       RB    24                     -0.6   
3      Patrick Mahomes       QB    37                    -13.4   
4   TreVeyon Henderson       RB    44                     -3.7   
5        Davante Adams       WR    57                      7.4   
6          Brock Purdy       QB    64                     -6.4   
7             DJ Moore       WR    77                      7.1   
8          Rashee Rice       WR    84                      5.0   
9     Jameson Williams       WR    97                      6.6   
10      Caleb Williams       QB   104                      0.4   
11        Tucker Kraft       TE   117                      1.9   
12        Emeka Egbuka       WR   124                      1.3   
13       Jaylen Warren       RB   137                      3.4   
14        Tyler Warren       TE   144                      1.7   

    Superflex Adjusted Value  
0                       -3.7  
1                       -6.4  
2                      -11.1  
3                       14.3  
4                      -12.0  
5                        2.2  
6                       10.8  
7                        3.6  
8                        1.5  
9                        3.9  
10                       9.4  
11                       0.4  
12                       0.2  
13                       2.3  
14                       1.0  , _body=<great_tables._gt_data.Body object at 0x7ef8c531d910>, _boxhead=Boxhead([ColInfo(var='Player', type=<ColInfoTypeEnum.stub: 2>, column_label='Player', column_align='left', column_width=None), ColInfo(var='Position', type=<ColInfoTypeEnum.default: 1>, column_label='Position', column_align='left', column_width=None), ColInfo(var='Pick', type=<ColInfoTypeEnum.default: 1>, column_label='Pick', column_align='right', column_width=None), ColInfo(var='Standard Adjusted Value', type=<ColInfoTypeEnum.default: 1>, column_label='Standard Adjusted Value', column_align='right', column_width=None), ColInfo(var='Superflex Adjusted Value', type=<ColInfoTypeEnum.default: 1>, column_label='Superflex Adjusted Value', column_align='right', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x7ef8c531d880>, _spanners=Spanners([]), _heading=Heading(title='Evaluating my draft picks using adjusted value', subtitle='Good value picks colored green, reaches colored red', preheader=None), _stubhead='Player', _source_notes=['Draft Data: ESPN Fantasy Football', 'Value Data: FantasyPros'], _footnotes=[], _styles=[StyleInfo(locname=LocBody(columns=None, rows=['TreVeyon Henderson'], mask=None), grpname=None, colname='Position', rownum=4, colnum=None, styles=[CellStyleFill(color='#f37b80')]), StyleInfo(locname=LocBody(columns=None, rows=['TreVeyon Henderson'], mask=None), grpname=None, colname='Pick', rownum=4, colnum=None, styles=[CellStyleFill(color='#f37b80')]), StyleInfo(locname=LocBody(columns=None, rows=['TreVeyon Henderson'], mask=None), grpname=None, colname='Standard Adjusted Value', rownum=4, colnum=None, styles=[CellStyleFill(color='#f37b80')]), StyleInfo(locname=LocBody(columns=None, rows=['TreVeyon Henderson'], mask=None), grpname=None, colname='Superflex Adjusted Value', rownum=4, colnum=None, styles=[CellStyleFill(color='#f37b80')]), StyleInfo(locname=LocBody(columns=None, rows=['DJ Moore', 'Jameson Williams', 'Jaylen Warren'], mask=None), grpname=None, colname='Position', rownum=7, colnum=None, styles=[CellStyleFill(color='#65ca8d')]), StyleInfo(locname=LocBody(columns=None, rows=['DJ Moore', 'Jameson Williams', 'Jaylen Warren'], mask=None), grpname=None, colname='Position', rownum=9, colnum=None, styles=[CellStyleFill(color='#65ca8d')]), StyleInfo(locname=LocBody(columns=None, rows=['DJ Moore', 'Jameson Williams', 'Jaylen Warren'], mask=None), grpname=None, colname='Position', rownum=13, 

In [ ]:
combined_df = pd.merge(standard_example, superflex_example, on='Team', how='inner')

In [ ]:
combined_value = combined_df.drop(columns=["Standard Adjusted Value", "Superflex Adjusted Value"])
combined_value

,Team,Standard Value,Superflex Value
0,Team 1,62,104
1,Team 2,-49,26
2,Team 3,39,93
3,Team 4,-49,-91
4,Team 5,127,86
5,Team 6,-30,-25
6,Team Kyle,196,202
7,Team 8,-405,-329
8,Team 9,8,81
9,Team 10,59,56


In [ ]:
combined_adj_value = combined_df.drop(columns=["Standard Value", "Superflex Value"])
combined_adj_value

,Team,Standard Adjusted Value,Superflex Adjusted Value
0,Team 1,-23.8,2.0
1,Team 2,-39.2,-11.2
2,Team 3,-11.9,-11.5
3,Team 4,-29.0,-24.2
4,Team 5,-20.1,-7.0
5,Team 6,0.3,-37.7
6,Team Kyle,15.3,16.4
7,Team 8,-79.6,-54.5
8,Team 9,-7.3,-6.8
9,Team 10,-7.6,3.3


In [ ]:
(
  GT(combined_value)
  .tab_header(
      title="How well did I draft?",
      subtitle="Fantasy football draft results based on standard and superflex value"
  )
  .tab_stub(rowname_col="Team")
  .tab_stubhead(label="Team")
  .fmt_number(columns="Superflex Value", decimals=0)
  .tab_source_note(source_note="Draft Data: ESPN Fantasy Football")
  .tab_source_note(source_note="Value Data: FantasyPros")
  .tab_style(style=style.fill(color="lightblue"), locations=loc.body(rows="Team Kyle"))
)

GT(_tbl_data=        Team  Standard Value  Superflex Value
0     Team 1              62              104
1     Team 2             -49               26
2     Team 3              39               93
3     Team 4             -49              -91
4     Team 5             127               86
5     Team 6             -30              -25
6  Team Kyle             196              202
7     Team 8            -405             -329
8     Team 9               8               81
9    Team 10              59               56, _body=<great_tables._gt_data.Body object at 0x7ef8ccefb590>, _boxhead=Boxhead([ColInfo(var='Team', type=<ColInfoTypeEnum.stub: 2>, column_label='Team', column_align='left', column_width=None), ColInfo(var='Standard Value', type=<ColInfoTypeEnum.default: 1>, column_label='Standard Value', column_align='right', column_width=None), ColInfo(var='Superflex Value', type=<ColInfoTypeEnum.default: 1>, column_label='Superflex Value', column_align='right', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x7ef8ccefaf00>, _spanners=Spanners([]), _heading=Heading(title='How well did I draft?', subtitle='Fantasy football draft results based on standard and superflex value', preheader=None), _stubhead='Team', _source_notes=['Draft Data: ESPN Fantasy Football', 'Value Data: FantasyPros'], _footnotes=[], _styles=[StyleInfo(locname=LocBody(columns=None, rows='Team Kyle', mask=None), grpname=None, colname='Standard Value', rownum=6, colnum=None, styles=[CellStyleFill(color='lightblue')]), StyleInfo(locname=LocBody(columns=None, rows='Team Kyle', mask=None), grpname=None, colname='Superflex Value', rownum=6, colnum=None, styles=[CellStyleFill(color='lightblue')])], _locale=<great_tables._gt_data.Locale object at 0x7ef8ccefb680>, _formats=[<great_tables._gt_data.FormatInfo object at 0x7ef8cdd47b60>], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_additional_css=OptionsInfo(scss=False, category='table', type='values', value=[]), table_font_names=OptionsInfo(scss=False, category='table', type='values', value=['-apple-system', 'BlinkMacSystemFont', 'Segoe UI', 'Roboto', 'Oxygen', 'Ubuntu', 'Cantarell', 'Helvetica Neue', 'Fira Sans', 'Droid Sans', 'Arial', 'sans-serif']), table_font_size=OptionsInfo(scss=True, category='table', type='px', value='16px'), table_font_weight=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_style=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_color=OptionsInfo(scss=True, category='table', type='value', value='#333333'), table_font_color_light=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_border_top_include=OptionsInfo(scss=False, category='table', type='boolean', value=True), table_border_top_style=OptionsInfo(scss=True, category='table', type='value', value='solid'), table_border_top_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_top_color=OptionsInfo(scss=True, category='table', type='value', value='#A8A8A8'), table_border_right_style=OptionsInfo(scss=True, category='table', type='value', value='none'), table_border_right_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_right_color=OptionsInfo(scss=True, category='table', type='value', value='#D3D3D3'), table_border_bottom_include=OptionsInfo(scss=False, category='table', type='boolean', value=T

In [ ]:
(
  GT(combined_adj_value)
  .tab_header(
      title="How well did I draft?",
      subtitle="Fantasy football draft results based on standard and superflex adjusted value"
  )
  .tab_stub(rowname_col="Team")
  .tab_stubhead(label="Team")
  .fmt_number(columns=["Standard Adjusted Value", "Superflex Adjusted Value"], decimals=1)
  .tab_source_note(source_note="Draft Data: ESPN Fantasy Football")
  .tab_source_note(source_note="Value Data: FantasyPros")
  .tab_style(style=style.fill(color="pink"), locations=loc.body(rows="Team Kyle"))
)

GT(_tbl_data=        Team  Standard Adjusted Value  Superflex Adjusted Value
0     Team 1                    -23.8                       2.0
1     Team 2                    -39.2                     -11.2
2     Team 3                    -11.9                     -11.5
3     Team 4                    -29.0                     -24.2
4     Team 5                    -20.1                      -7.0
5     Team 6                      0.3                     -37.7
6  Team Kyle                     15.3                      16.4
7     Team 8                    -79.6                     -54.5
8     Team 9                     -7.3                      -6.8
9    Team 10                     -7.6                       3.3, _body=<great_tables._gt_data.Body object at 0x7e8a235675f0>, _boxhead=Boxhead([ColInfo(var='Team', type=<ColInfoTypeEnum.stub: 2>, column_label='Team', column_align='left', column_width=None), ColInfo(var='Standard Adjusted Value', type=<ColInfoTypeEnum.default: 1>, column_label='Standard Adjusted Value', column_align='right', column_width=None), ColInfo(var='Superflex Adjusted Value', type=<ColInfoTypeEnum.default: 1>, column_label='Superflex Adjusted Value', column_align='right', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x7e8a23558140>, _spanners=Spanners([]), _heading=Heading(title='How well did I draft?', subtitle='Fantasy football draft results based on standard and superflex adjusted value', preheader=None), _stubhead='Team', _source_notes=['Draft Data: ESPN Fantasy Football', 'Value Data: FantasyPros'], _footnotes=[], _styles=[StyleInfo(locname=LocBody(columns=None, rows='Team Kyle', mask=None), grpname=None, colname='Standard Adjusted Value', rownum=6, colnum=None, styles=[CellStyleFill(color='pink')]), StyleInfo(locname=LocBody(columns=None, rows='Team Kyle', mask=None), grpname=None, colname='Superflex Adjusted Value', rownum=6, colnum=None, styles=[CellStyleFill(color='pink')])], _locale=<great_tables._gt_data.Locale object at 0x7e8a23558470>, _formats=[<great_tables._gt_data.FormatInfo object at 0x7e8a235648c0>], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_additional_css=OptionsInfo(scss=False, category='table', type='values', value=[]), table_font_names=OptionsInfo(scss=False, category='table', type='values', value=['-apple-system', 'BlinkMacSystemFont', 'Segoe UI', 'Roboto', 'Oxygen', 'Ubuntu', 'Cantarell', 'Helvetica Neue', 'Fira Sans', 'Droid Sans', 'Arial', 'sans-serif']), table_font_size=OptionsInfo(scss=True, category='table', type='px', value='16px'), table_font_weight=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_style=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_color=OptionsInfo(scss=True, category='table', type='value', value='#333333'), table_font_color_light=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_border_top_include=OptionsInfo(scss=False, category='table', type='boolean', value=True), table_border_top_style=OptionsInfo(scss=True, category='table', type='value', value='solid'), table_border_top_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_top_color=OptionsInfo(scss=True, category='table', type='value', value='#A8A8A8'), table_border_right_style=OptionsInfo(scss=True, category='table', type='value', value='none'), table_border_right_width=Option